# Desafio: Consumo de Dados para Previsão do Tempo das Cidades do Vale do Paraíba.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Neste desafio, você desenvolverá um notebook que será responsável por extrair dados de previsão do tempo das cidades do Vale do Paraíba, região onde se localiza a Dataside. Para consultar todas as cidades dessa região, utilizaremos a API do IBGE. No caso, basta realizar uma requisição HTTP com o método GET, utilizando a URL abaixo:

```
https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios
```

Com esses dados, gerar um data frame e a partir dele uma temp view. Ex: "cities"

Utilizando os nomes das cidades, deverão ser consultados os dados de previsão de tempo para cada cidade. Para realizar essa consulta, poderá ser utilizada qualquer uma das APIs informadas no link abaixo.

[Public APIs - Wather](https://github.com/public-apis/public-apis#weather)

Obs.: Para algumas, pode ser necessário cadastrar-se para acessar sua API Key. Mas nenhuma delas deve precisar cadastrar cartão de crédito ou adicionar qualquer valor monetário para utilizar. Caso alguma solicite, basta optar por outra.

Com os dados consultados, gerar um data frame e partir dele outra temp view. Ex: "forecasts"

Com as temp views geradas, utilizar Spark SQL para criar queries e gerar data frames das seguintes tabelas:

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento
    
    Obs.: Os valores da coluna "VaiChover" deverá ser "Sim" ou "Não". E a coluna "CodigoDaCidade" é o ID retornado junto com os nomes da cidades na API do IBGE.
    Obs.: Dependendo da API utilizada, algumas colunas podem não existir e ficarão em branco. Você deve optar por uma API que traga o maior número de informações possível.

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

Essas tabelas deverão ser exportadas em formado CSV e entregue no final do desafio.

## To Do

[ ] - Consultar municípios do Vale do Paraíba, gerar um data frame e criar uma temp view com esses dados.
[ ] - Consultar dados do tempo para cada município, gerar um data frame e criar uma outra temp view.
[ ] - Utilizar Spark SQL para gerar os data frames das Tabelas 1 e 2.
[ ] - Exportar os data frames para CSV.

## Atenção

- Existe um limite de requisições de 10000 requests por conta cadastrada na m3o.
- Essa API pode retornar cidades de outras regiões que possuem nome semelhante a alguma cidade do Vale do Paraiba. Pode mantê-las ou filtrar para gerar as tabelas apenas com dados de Regiao = Sao Paulo. Fica a seu critério.

## Entregando o desafio

Concluindo todos os passos informados em To Do, basta salvar o arquivo .ipynb do notebook e enviar para a Dataside juntamente com os CSVs das duas tabelas.


In [4]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 51.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=31f26ce49fca4bb083ef0e81fc264ee7268f93458bfc821efa994db740d8e4b8
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [5]:
pip install findspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 7.4 MB/s 


In [22]:
# Importanto bibliotecas e iniciando findspark
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql import functions as f

# Definindo parâmetros com o SparkConf 
# SparkConf seta um nome para a aplicação e a URL para o meu cluster distribuído 
spark_conf = SparkConf() \
      .setAppName("dataside") \
      .setMaster("local[2]")

sc = SparkContext.getOrCreate();

spark = SparkSession.builder \
      .master("local[2]") \
      .appName("dataside") \
      .getOrCreate()

In [ ]:
# Buscando cidades do Vale do Paraíba usando a API do IBGE indicada

# 1. Realizando a requsição HTTP usando o método get
r = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios") 

cidades = r.json()

## Mostrando os dados em forma não estruturada - JSON
cidades

In [35]:
# 2. Criar data frame com as cidades

dataframe = spark.read.json(sc.parallelize(cidades))

dataframe.show(dataframe.count(), truncate=False)

+-------+------------------------------------------------------------------------------------------------------+-----------------------+---------------------------------------------------------------------------------------------------------------------+
|id     |microrregiao                                                                                          |nome                   |regiao-imediata                                                                                                      |
+-------+------------------------------------------------------------------------------------------------------+-----------------------+---------------------------------------------------------------------------------------------------------------------+
|3502507|{35051, {{35, São Paulo, {3, Sudeste, SE}, SP}, 3513, Vale do Paraíba Paulista}, Guaratinguetá}       |Aparecida              |{350052, Guaratinguetá, {{35, São Paulo, {3, Sudeste, SE}, SP}, 3511, São José dos Campos}}        

In [64]:
# 3. Criar view com as cidades

dataframe.createOrReplaceTempView('cidades')
# todas_cidades = spark.sql("SELECT * from cidades")
# spark.sql(todas_cidades).toPandas()

nomes_cidades = spark.sql("SELECT nome FROM cidades")
nomes_cidades.toPandas()

,nome
0,Aparecida
1,Arapeí
2,Areias
3,Bananal
4,Caçapava
5,Cachoeira Paulista
6,Campos do Jordão
7,Canas
8,Caraguatatuba
9,Cruzeiro


In [65]:
# Criando lista com o nome de todas as cidades
lista_cidades = dataframe.select("nome").rdd.flatMap(lambda x: x).collect()
lista_cidades

['Aparecida',
 'Arapeí',
 'Areias',
 'Bananal',
 'Caçapava',
 'Cachoeira Paulista',
 'Campos do Jordão',
 'Canas',
 'Caraguatatuba',
 'Cruzeiro',
 'Cunha',
 'Guaratinguetá',
 'Igaratá',
 'Ilhabela',
 'Jacareí',
 'Jambeiro',
 'Lagoinha',
 'Lavrinhas',
 'Lorena',
 'Monteiro Lobato',
 'Natividade da Serra',
 'Paraibuna',
 'Pindamonhangaba',
 'Piquete',
 'Potim',
 'Queluz',
 'Redenção da Serra',
 'Roseira',
 'Santa Branca',
 'Santo Antônio do Pinhal',
 'São Bento do Sapucaí',
 'São José do Barreiro',
 'São José dos Campos',
 'São Luiz do Paraitinga',
 'São Sebastião',
 'Silveiras',
 'Taubaté',
 'Tremembé',
 'Ubatuba']

In [11]:
# 4. Buscar previsão do tempo para as cidades
# TODO



In [12]:
# 5. Criar data frame com as previsões
# TODO



In [13]:
# 6. Criar view com as previsões



In [14]:
# Criar DF da Tabela 1
# TODO

In [15]:
# Criar DF da Tabela 2
# TODO

In [16]:
# Exportar CSVs
# TODO